# 🎧 Platform Penalty Analysis

**Simulate how your masters sound on different streaming platforms:**
- 🎵 **Spotify**: Loudness normalization (-14 LUFS) + MP3 compression artifacts
- 🎬 **YouTube**: Loudness normalization (-14 LUFS) + aggressive compression + stereo limiting
- 🍎 **Apple Music**: Loudness normalization (-16 LUFS) + AAC compression + gentle limiting
- 🎧 **SoundCloud**: MP3 compression + dynamic range reduction + high-frequency rolloff
- 📻 **Radio**: Heavy compression + stereo narrowing + EQ curve + brick-wall limiting
- 💿 **CD**: No streaming artifacts, just format conversion simulation

**Features:**
- Batch process all masters in a folder
- Platform-specific audio processing simulation
- Organized output with platform folders and suffixes
- Before/after analysis reports

## ⚙️ User Configuration
**Edit these settings, then run all cells**

In [1]:
# ============================================================================
# 🎧 PLATFORM PENALTY CONFIGURATION
# ============================================================================

# 📁 INPUT SETTINGS
# Path to folder containing master WAV files
MASTERS_FOLDER_PATH = "/Users/itay/Documents/post_mix_data/Masters/"

# 🎵 PLATFORM SELECTION
# Choose which platforms to simulate (True = enable, False = skip)
PLATFORMS = {
    'spotify': True,      # Spotify streaming simulation
    'youtube': True,      # YouTube streaming simulation  
    'apple_music': False,  # Apple Music streaming simulation
    'soundcloud': False,   # SoundCloud streaming simulation
    'radio': False,        # Commercial radio simulation
    'cd': False,           # CD format simulation
}

# 🎚️ PROCESSING OPTIONS
APPLY_LOUDNESS_NORMALIZATION = True    # Apply platform loudness targets
SIMULATE_COMPRESSION_ARTIFACTS = True  # Simulate MP3/AAC artifacts
APPLY_PLATFORM_EQ = True              # Apply platform-specific EQ curves
CREATE_ANALYSIS_REPORT = True         # Generate before/after analysis

# 🎧 OUTPUT SETTINGS
OUTPUT_BASE_DIR = "/Users/itay/Documents/post_mix_data"  # Base output directory
OUTPUT_FOLDER_NAME = "penalized_masters"                # Output folder name
OUTPUT_FORMAT = "PCM_24"                               # Output bit depth

print("✅ Platform penalty configuration loaded!")
print(f"📁 Masters folder: {MASTERS_FOLDER_PATH}")
print(f"🎵 Platforms enabled: {', '.join([p for p, enabled in PLATFORMS.items() if enabled])}")
print(f"📊 Analysis report: {'Enabled' if CREATE_ANALYSIS_REPORT else 'Disabled'}")

✅ Platform penalty configuration loaded!
📁 Masters folder: /Users/itay/Documents/post_mix_data/Masters/
🎵 Platforms enabled: spotify, youtube
📊 Analysis report: Enabled


## 📦 Load Dependencies

In [2]:
# Core imports
import os
import json
import numpy as np
import soundfile as sf
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Audio processing
from dsp_premitives import (
    peaking_eq, shelf_filter, highpass_filter, lowpass_filter,
    compressor, normalize_lufs, measure_peak, measure_rms,
    stereo_widener, apply_gain_db, _db_to_lin, _lin_to_db
)

# Analysis (if available)
try:
    from analysis import analyze_wav
    ANALYSIS_AVAILABLE = True
except ImportError:
    ANALYSIS_AVAILABLE = False
    print("⚠️ Analysis module not available - skipping detailed analysis")

print("🎧 Platform Penalty System Loaded!")
print("✅ Audio processing capabilities ready")
print("🚀 Ready to simulate streaming platforms!")

DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
🎧 Platform Penalty System Loaded!
✅ Audio processing capabilities ready
🚀 Ready to simulate streaming platforms!


## 🎵 Platform Processing Engines

In [3]:
class PlatformProcessor:
    """Base class for platform-specific audio processing"""
    
    def __init__(self, platform_name: str, target_lufs: float = -14.0, can_boost: bool = False, boost_headroom_db: float = 1.0):
        self.platform_name = platform_name
        self.target_lufs = target_lufs
        self.can_boost = can_boost
        self.boost_headroom_db = boost_headroom_db
        self.sample_rate = 44100
    
    def process(self, audio: np.ndarray, sr: int) -> np.ndarray:
        """Apply platform-specific processing"""
        self.sample_rate = sr
        processed = audio.copy()
        
        # Apply platform-specific processing
        processed = self._apply_platform_processing(processed)
        
        # Apply loudness normalization if enabled
        if APPLY_LOUDNESS_NORMALIZATION:
            processed = self._apply_loudness_normalization(processed)
        
        return processed
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """Override in subclasses for platform-specific processing"""
        return audio
    
    def _apply_loudness_normalization(self, audio: np.ndarray) -> np.ndarray:
        """Apply loudness normalization based on platform behavior"""
        try:
            # Measure current loudness
            from dsp_premitives import measure_lufs
            current_lufs = measure_lufs(audio, self.sample_rate)
            
            # Check peak before normalization
            peak_before = np.max(np.abs(audio))
            peak_db_before = 20 * np.log10(peak_before) if peak_before > 0 else -100
            
            # Calculate loudness difference
            loudness_diff = self.target_lufs - current_lufs
            
            if loudness_diff < 0:
                # Song is louder than target - apply penalty (turn it down)
                gain_db = loudness_diff
                gain_linear = 10 ** (gain_db / 20)
                
                processed = audio * gain_linear
                peak_after = np.max(np.abs(processed))
                peak_db_after = 20 * np.log10(peak_after) if peak_after > 0 else -100
                
                print(f"      📉 Loudness Penalty: {gain_db:.1f} dB (LUFS: {current_lufs:.1f} → {self.target_lufs})")
                print(f"         Peak: {peak_db_before:.1f} → {peak_db_after:.1f} dBFS (should never increase!)")
                
                return processed
                
            elif loudness_diff > 0 and self.can_boost:
                # Song is quieter and platform can boost
                # Apply boost but leave headroom to prevent clipping
                max_boost = loudness_diff - self.boost_headroom_db
                if max_boost > 0:
                    gain_db = max_boost
                    gain_linear = 10 ** (gain_db / 20)
                    
                    # Check if boost would cause clipping
                    peak_after_boost = peak_before * gain_linear
                    if peak_after_boost > 0.95:  # Leave some headroom
                        # Limit boost to prevent clipping
                        gain_linear = 0.95 / peak_before
                        gain_db = 20 * np.log10(gain_linear)
                    
                    processed = audio * gain_linear
                    peak_after = np.max(np.abs(processed))
                    peak_db_after = 20 * np.log10(peak_after) if peak_after > 0 else -100
                    
                    print(f"      📈 Volume Boost: +{gain_db:.1f} dB (LUFS: {current_lufs:.1f} → ~{current_lufs + gain_db:.1f})")
                    print(f"         Peak: {peak_db_before:.1f} → {peak_db_after:.1f} dBFS (max: -0.4 dBFS)")
                    
                    # Final safety check - hard clip prevention
                    if peak_after > 1.0:
                        print(f"         ⚠️ CLIPPING DETECTED! Reducing gain...")
                        processed = processed * (0.99 / peak_after)
                    
                    return processed
                else:
                    print(f"      ✅ No change (too close to target)")
                    return audio
            else:
                # Song is quieter but platform doesn't boost
                print(f"      ✅ No boost (LUFS: {current_lufs:.1f}, platform doesn't boost)")
                print(f"         Peak: {peak_db_before:.1f} dBFS")
                return audio
                
        except Exception as e:
            print(f"      ⚠️ Could not measure LUFS: {e}")
            # Fallback: assume it's a loud master and apply typical penalty
            return audio * 0.8  # -2dB typical penalty

In [4]:
class SpotifyProcessor(PlatformProcessor):
    """Spotify streaming simulation (Normal mode)"""
    
    def __init__(self):
        # Spotify can boost quiet tracks, leaving ~1dB headroom
        super().__init__("Spotify", target_lufs=-14.0, can_boost=True, boost_headroom_db=1.0)
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """Spotify: Clean normalization + mild MP3 artifacts"""
        processed = audio.copy()
        
        if SIMULATE_COMPRESSION_ARTIFACTS:
            # Simulate mild MP3 compression artifacts
            # High frequency rolloff only - very gentle
            processed = lowpass_filter(processed, self.sample_rate, 17000, order=1)
        
        return processed


class YouTubeProcessor(PlatformProcessor):
    """YouTube streaming simulation"""
    
    def __init__(self):
        # YouTube only turns down, never boosts
        super().__init__("YouTube", target_lufs=-14.0, can_boost=False)
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """YouTube: Compression artifacts + codec simulation"""
        processed = audio.copy()
        
        if SIMULATE_COMPRESSION_ARTIFACTS:
            # High frequency rolloff for codec simulation
            processed = lowpass_filter(processed, self.sample_rate, 16000, order=1)
            
            # NO compression - just codec artifacts
            # YouTube's actual loudness normalization doesn't add compression
            # It just turns down the volume
            
            # Very slight stereo width reduction for codec
            if audio.ndim == 2:
                processed = stereo_widener(processed, width=0.95)
        
        return processed

In [5]:
class AppleMusicProcessor(PlatformProcessor):
    """Apple Music streaming simulation"""
    
    def __init__(self):
        super().__init__("Apple Music", target_lufs=-16.0)
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """Apple Music: High quality AAC + gentle processing"""
        processed = audio.copy()
        
        if SIMULATE_COMPRESSION_ARTIFACTS:
            # High quality AAC simulation - minimal artifacts
            processed = lowpass_filter(processed, self.sample_rate, 18000, order=1)
            
            # Very gentle limiting
            processed = compressor(
                processed, self.sample_rate,
                threshold_db=-3.0,
                ratio=2.0,
                attack_ms=5.0,
                release_ms=100.0
            )
        
        return processed


class SoundCloudProcessor(PlatformProcessor):
    """SoundCloud streaming simulation"""
    
    def __init__(self):
        super().__init__("SoundCloud", target_lufs=-14.0)
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """SoundCloud: Heavy MP3 compression + processing"""
        processed = audio.copy()
        
        if SIMULATE_COMPRESSION_ARTIFACTS:
            # Heavy MP3 compression artifacts
            processed = lowpass_filter(processed, self.sample_rate, 11000, order=4)
            
            # Dynamic range reduction
            processed = compressor(
                processed, self.sample_rate,
                threshold_db=-12.0,
                ratio=4.0,
                attack_ms=2.0,
                release_ms=50.0
            )
            
            # Additional harmonic distortion
            processed = np.tanh(processed * 1.1) / 1.1
        
        return processed

In [6]:
class RadioProcessor(PlatformProcessor):
    """Commercial radio simulation"""
    
    def __init__(self):
        super().__init__("Radio", target_lufs=-12.0)
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """Radio: Heavy compression + EQ + stereo narrowing"""
        processed = audio.copy()
        
        # Radio EQ curve
        if APPLY_PLATFORM_EQ:
            processed = highpass_filter(processed, self.sample_rate, 80, order=2)  # Remove sub
            processed = peaking_eq(processed, self.sample_rate, 100, -2.0, 1.0)    # Reduce mud
            processed = peaking_eq(processed, self.sample_rate, 3000, 2.0, 1.5)    # Presence boost
            processed = peaking_eq(processed, self.sample_rate, 8000, 1.5, 1.0)    # Air
            processed = lowpass_filter(processed, self.sample_rate, 13000, order=2) # Top rolloff
        
        # Heavy compression
        processed = compressor(
            processed, self.sample_rate,
            threshold_db=-6.0,
            ratio=6.0,
            attack_ms=0.5,
            release_ms=20.0
        )
        
        # Stereo narrowing for mono compatibility
        if audio.ndim == 2:
            processed = stereo_widener(processed, width=0.6)
        
        # Brick wall limiting
        processed = compressor(
            processed, self.sample_rate,
            threshold_db=-1.0,
            ratio=20.0,
            attack_ms=0.1,
            release_ms=5.0
        )
        
        return processed


class CDProcessor(PlatformProcessor):
    """CD format simulation"""
    
    def __init__(self):
        super().__init__("CD", target_lufs=-16.0)
    
    def _apply_platform_processing(self, audio: np.ndarray) -> np.ndarray:
        """CD: Minimal processing - format conversion only"""
        processed = audio.copy()
        
        # CD format simulation: 16-bit quantization noise simulation
        if SIMULATE_COMPRESSION_ARTIFACTS:
            # Add subtle quantization noise
            noise_floor = 1.0 / (2**16)  # 16-bit noise floor
            noise = np.random.random(processed.shape) * noise_floor * 0.1
            processed = processed + noise
        
        return processed

In [7]:
# Platform processor factory
PLATFORM_PROCESSORS = {
    'spotify': SpotifyProcessor,
    'youtube': YouTubeProcessor,
    'apple_music': AppleMusicProcessor,
    'soundcloud': SoundCloudProcessor,
    'radio': RadioProcessor,
    'cd': CDProcessor,
}

print("🎵 Platform processors loaded:")
for platform in PLATFORM_PROCESSORS.keys():
    status = "✅" if PLATFORMS.get(platform, False) else "⏸️"
    print(f"  {status} {platform.title().replace('_', ' ')}")

🎵 Platform processors loaded:
  ✅ Spotify
  ✅ Youtube
  ⏸️ Apple Music
  ⏸️ Soundcloud
  ⏸️ Radio
  ⏸️ Cd


## 📁 Discover Master Files

In [8]:
def find_master_files(masters_folder: str) -> list:
    """Find all WAV files in the masters folder"""
    masters_path = Path(masters_folder)
    
    if not masters_path.exists():
        raise FileNotFoundError(f"Masters folder not found: {masters_folder}")
    
    # Find all WAV files
    wav_files = []
    for wav_file in masters_path.rglob('*.wav'):
        if wav_file.is_file():
            wav_files.append(wav_file)
    
    # Sort by name for consistent processing order
    wav_files.sort(key=lambda x: x.name)
    
    return wav_files


# Discover master files
print(f"🔍 Scanning for master files in: {MASTERS_FOLDER_PATH}")

try:
    master_files = find_master_files(MASTERS_FOLDER_PATH)
    
    print(f"\n📊 Found {len(master_files)} master files:")
    for i, master_file in enumerate(master_files, 1):
        relative_path = os.path.relpath(master_file, MASTERS_FOLDER_PATH)
        print(f"  {i:2d}. {relative_path}")
    
    if len(master_files) == 0:
        print("❌ No WAV files found in the specified folder!")
    else:
        print(f"\n✅ Ready to process {len(master_files)} files")
        
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    master_files = []

🔍 Scanning for master files in: /Users/itay/Documents/post_mix_data/Masters/

📊 Found 2 master files:
   1. Bass_Heavy.wav
   2. Stem_Aggressive.wav

✅ Ready to process 2 files


## 🎧 Setup Output Structure

In [9]:
# Create output directory structure
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# Create mother folder and timestamped subfolder
mother_folder = Path(OUTPUT_BASE_DIR) / OUTPUT_FOLDER_NAME
mother_folder.mkdir(parents=True, exist_ok=True)
output_base = mother_folder / f"session_{timestamp}"

print(f"📁 Creating output structure...")
print(f"   Mother folder: {mother_folder}")
print(f"   Session folder: {output_base}")

# Create base output directory
output_base.mkdir(parents=True, exist_ok=True)

# Create platform subdirectories
platform_dirs = {}
enabled_platforms = [platform for platform, enabled in PLATFORMS.items() if enabled]

for platform in enabled_platforms:
    platform_dir = output_base / platform
    platform_dir.mkdir(exist_ok=True)
    platform_dirs[platform] = platform_dir
    print(f"   ✅ {platform}: {platform_dir}")

# Create analysis directory if needed
if CREATE_ANALYSIS_REPORT:
    analysis_dir = output_base / "analysis"
    analysis_dir.mkdir(exist_ok=True)
    print(f"   📊 analysis: {analysis_dir}")

print(f"\n✅ Output structure ready!")
print(f"📂 {len(platform_dirs)} platform folders created")

📁 Creating output structure...
   Mother folder: /Users/itay/Documents/post_mix_data/penalized_masters
   Session folder: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200
   ✅ spotify: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200/spotify
   ✅ youtube: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200/youtube
   📊 analysis: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200/analysis

✅ Output structure ready!
📂 2 platform folders created


## 🎛️ Process Masters for All Platforms

In [10]:
if master_files and enabled_platforms:
    print(f"🎛️ PROCESSING {len(master_files)} MASTERS FOR {len(enabled_platforms)} PLATFORMS")
    print("=" * 70)
    
    # Initialize processors
    processors = {}
    for platform in enabled_platforms:
        processors[platform] = PLATFORM_PROCESSORS[platform]()
    
    # Process each master file
    total_files_processed = 0
    processing_results = []
    
    for file_idx, master_file in enumerate(master_files, 1):
        print(f"\n[{file_idx}/{len(master_files)}] Processing: {master_file.name}")
        
        try:
            # Load master file
            audio_data, sample_rate = sf.read(str(master_file))
            
            print(f"   📊 Loaded: {audio_data.shape} @ {sample_rate} Hz")
            
            # Get base filename without extension
            base_name = master_file.stem
            
            file_results = {
                'original_file': str(master_file),
                'base_name': base_name,
                'platforms': {}
            }
            
            # Process for each enabled platform
            for platform in enabled_platforms:
                print(f"   🎵 Processing for {platform.title().replace('_', ' ')}...")
                
                try:
                    # Apply platform processing
                    processor = processors[platform]
                    processed_audio = processor.process(audio_data, sample_rate)
                    
                    # Create output filename with platform suffix
                    output_filename = f"{base_name}_{platform}.wav"
                    output_path = platform_dirs[platform] / output_filename
                    
                    # Save processed audio
                    sf.write(str(output_path), processed_audio, sample_rate, subtype=OUTPUT_FORMAT)
                    
                    # Calculate basic metrics
                    original_peak = measure_peak(audio_data)
                    processed_peak = measure_peak(processed_audio)
                    original_rms = measure_rms(audio_data)
                    processed_rms = measure_rms(processed_audio)
                    
                    file_results['platforms'][platform] = {
                        'output_path': str(output_path),
                        'original_peak_db': _lin_to_db(original_peak),
                        'processed_peak_db': _lin_to_db(processed_peak),
                        'original_rms_db': _lin_to_db(original_rms),
                        'processed_rms_db': _lin_to_db(processed_rms),
                        'peak_change_db': _lin_to_db(processed_peak) - _lin_to_db(original_peak),
                        'rms_change_db': _lin_to_db(processed_rms) - _lin_to_db(original_rms)
                    }
                    
                    print(f"      ✅ Saved: {output_filename}")
                    print(f"      📊 Peak: {_lin_to_db(original_peak):.1f} → {_lin_to_db(processed_peak):.1f} dBFS")
                    
                    total_files_processed += 1
                    
                except Exception as e:
                    print(f"      ❌ Error processing {platform}: {e}")
                    file_results['platforms'][platform] = {'error': str(e)}
            
            processing_results.append(file_results)
            
        except Exception as e:
            print(f"   ❌ Error loading file: {e}")
            processing_results.append({
                'original_file': str(master_file),
                'base_name': master_file.stem,
                'error': str(e)
            })
    
    print("\n" + "=" * 70)
    print("🏆 PLATFORM PROCESSING COMPLETE!")
    print("=" * 70)
    print(f"📊 SUMMARY:")
    print(f"  • Masters processed: {len(master_files)}")
    print(f"  • Platforms: {len(enabled_platforms)}")
    print(f"  • Total files created: {total_files_processed}")
    print(f"  • Output location: {output_base}")
    
    print(f"\n📁 PLATFORM BREAKDOWN:")
    for platform in enabled_platforms:
        platform_file_count = len([f for f in platform_dirs[platform].glob('*.wav')])
        print(f"  • {platform.title().replace('_', ' ')}: {platform_file_count} files")

elif not master_files:
    print("❌ No master files found - cannot process")
    
elif not enabled_platforms:
    print("❌ No platforms enabled - cannot process")
    
else:
    print("❌ Cannot process - check configuration")

🎛️ PROCESSING 2 MASTERS FOR 2 PLATFORMS

[1/2] Processing: Bass_Heavy.wav
   📊 Loaded: (15787800, 2) @ 44100 Hz
   🎵 Processing for Spotify...
      ⚠️ Could not measure LUFS: cannot import name 'measure_lufs' from 'dsp_premitives' (/Users/itay/Documents/GitHub/post_mix_analysis/dsp_premitives.py)
      ✅ Saved: Bass_Heavy_spotify.wav
      📊 Peak: 0.0 → -1.7 dBFS
   🎵 Processing for Youtube...
      ⚠️ Could not measure LUFS: cannot import name 'measure_lufs' from 'dsp_premitives' (/Users/itay/Documents/GitHub/post_mix_analysis/dsp_premitives.py)
      ✅ Saved: Bass_Heavy_youtube.wav
      📊 Peak: 0.0 → -1.7 dBFS

[2/2] Processing: Stem_Aggressive.wav
   📊 Loaded: (15787800, 2) @ 44100 Hz
   🎵 Processing for Spotify...
      ⚠️ Could not measure LUFS: cannot import name 'measure_lufs' from 'dsp_premitives' (/Users/itay/Documents/GitHub/post_mix_analysis/dsp_premitives.py)
      ✅ Saved: Stem_Aggressive_spotify.wav
      📊 Peak: 0.0 → -1.6 dBFS
   🎵 Processing for Youtube...
      ⚠️ C

## 📊 Generate Analysis Report

In [11]:
if CREATE_ANALYSIS_REPORT and 'processing_results' in locals() and processing_results:
    print("📊 GENERATING ANALYSIS REPORT")
    print("=" * 50)
    
    # Create analysis summary
    analysis_summary = {
        'timestamp': datetime.now().isoformat(),
        'masters_folder': MASTERS_FOLDER_PATH,
        'output_folder': str(output_base),
        'platforms_processed': enabled_platforms,
        'total_masters': len(master_files),
        'total_files_created': total_files_processed,
        'configuration': {
            'loudness_normalization': APPLY_LOUDNESS_NORMALIZATION,
            'compression_artifacts': SIMULATE_COMPRESSION_ARTIFACTS,
            'platform_eq': APPLY_PLATFORM_EQ,
            'output_format': OUTPUT_FORMAT
        },
        'processing_results': processing_results
    }
    
    # Save JSON report
    report_path = analysis_dir / f"platform_penalty_report_{timestamp}.json"
    with open(report_path, 'w') as f:
        json.dump(analysis_summary, f, indent=2)
    
    print(f"✅ JSON report saved: {report_path}")
    
    # Generate text summary
    summary_path = analysis_dir / f"platform_penalty_summary_{timestamp}.txt"
    with open(summary_path, 'w') as f:
        f.write("🎧 PLATFORM PENALTY ANALYSIS SUMMARY\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Masters folder: {MASTERS_FOLDER_PATH}\n")
        f.write(f"Output folder: {output_base}\n\n")
        
        f.write(f"📊 PROCESSING SUMMARY:\n")
        f.write(f"  • Masters processed: {len(master_files)}\n")
        f.write(f"  • Platforms: {', '.join(enabled_platforms)}\n")
        f.write(f"  • Total files created: {total_files_processed}\n\n")
        
        f.write(f"🎵 PLATFORM BREAKDOWN:\n")
        for platform in enabled_platforms:
            platform_file_count = len([f for f in platform_dirs[platform].glob('*.wav')])
            f.write(f"  • {platform.title().replace('_', ' ')}: {platform_file_count} files\n")
        
        f.write(f"\n📁 OUTPUT STRUCTURE:\n")
        for platform in enabled_platforms:
            f.write(f"  📂 {platform}/\n")
            for wav_file in sorted(platform_dirs[platform].glob('*.wav')):
                f.write(f"     • {wav_file.name}\n")
        
        # Add level change analysis
        f.write(f"\n📈 LEVEL CHANGES SUMMARY:\n")
        for result in processing_results:
            if 'error' not in result and 'platforms' in result:
                f.write(f"\n  🎵 {result['base_name']}:\n")
                for platform, metrics in result['platforms'].items():
                    if 'error' not in metrics:
                        peak_change = metrics['peak_change_db']
                        rms_change = metrics['rms_change_db']
                        f.write(f"    • {platform.title().replace('_', ' ')}: "
                               f"Peak {peak_change:+.1f}dB, RMS {rms_change:+.1f}dB\n")
    
    print(f"✅ Text summary saved: {summary_path}")
    
    print(f"\n📊 ANALYSIS COMPLETE!")
    print(f"   Reports saved in: {analysis_dir}")

elif CREATE_ANALYSIS_REPORT:
    print("⚠️ Analysis report requested but no processing results available")
else:
    print("ℹ️ Analysis report generation disabled")

📊 GENERATING ANALYSIS REPORT
✅ JSON report saved: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200/analysis/platform_penalty_report_20250902_101200.json
✅ Text summary saved: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200/analysis/platform_penalty_summary_20250902_101200.txt

📊 ANALYSIS COMPLETE!
   Reports saved in: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200/analysis


## 🎉 Summary

In [12]:
# Final summary
print("=" * 70)
print("🎉 PLATFORM PENALTY ANALYSIS COMPLETE!")
print("=" * 70)

if 'master_files' in locals() and master_files and enabled_platforms:
    print(f"📁 Output: {output_base}")
    print(f"🎵 Masters processed: {len(master_files)}")
    print(f"🎧 Platforms simulated: {', '.join(enabled_platforms)}")
    print(f"📊 Total penalized files created: {total_files_processed}")
    
    print(f"\n📂 FOLDER STRUCTURE:")
    print(f"   {output_base.name}/")
    for platform in enabled_platforms:
        file_count = len([f for f in platform_dirs[platform].glob('*.wav')])
        print(f"   ├── {platform}/ ({file_count} files)")
        
        # Show first few files as examples
        example_files = list(platform_dirs[platform].glob('*.wav'))[:3]
        for i, example_file in enumerate(example_files):
            prefix = "│   ├──" if i < len(example_files) - 1 else "│   └──"
            print(f"   {prefix} {example_file.name}")
        
        if file_count > 3:
            print(f"   │   └── ... ({file_count - 3} more files)")
    
    if CREATE_ANALYSIS_REPORT:
        print(f"   └── analysis/ (reports)")
    
    print(f"\n✨ PLATFORM PROCESSING APPLIED:")
    for platform in enabled_platforms:
        processor = processors[platform]
        target_lufs = processor.target_lufs
        print(f"  • {platform.title().replace('_', ' ')}: {target_lufs} LUFS normalization + platform-specific processing")
    
    print(f"\n🎧 Ready for A/B testing against originals!")
    print(f"🔍 Compare how your masters sound on different platforms")
    
else:
    print("❌ Processing was not completed")
    print("   Check configuration and input paths")

print("\n" + "=" * 70)

🎉 PLATFORM PENALTY ANALYSIS COMPLETE!
📁 Output: /Users/itay/Documents/post_mix_data/penalized_masters/session_20250902_101200
🎵 Masters processed: 2
🎧 Platforms simulated: spotify, youtube
📊 Total penalized files created: 4

📂 FOLDER STRUCTURE:
   session_20250902_101200/
   ├── spotify/ (2 files)
   │   ├── Stem_Aggressive_spotify.wav
   │   └── Bass_Heavy_spotify.wav
   ├── youtube/ (2 files)
   │   ├── Bass_Heavy_youtube.wav
   │   └── Stem_Aggressive_youtube.wav
   └── analysis/ (reports)

✨ PLATFORM PROCESSING APPLIED:
  • Spotify: -14.0 LUFS normalization + platform-specific processing
  • Youtube: -14.0 LUFS normalization + platform-specific processing

🎧 Ready for A/B testing against originals!
🔍 Compare how your masters sound on different platforms

